In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME", "STATUS"], axis=1)

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Determine a cutoff value based on the number of data points for each unique value
cutoff = 150

# Create a list of application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()

# Replace the specified application types with "Other" in the DataFrame
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(
    lambda x: "Other" if x in application_types_to_replace else x
)


In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts_greater_than_one = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_one)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 150

# Create a list of classifications to be replaced with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace the specified classifications with "Other" in the DataFrame
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop("IS_SUCCESSFUL", axis=1)
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define a function to create and train a deep neural network
def create_and_train_model(hidden_layer_sizes=(128, 64), alpha=0.0001, dropout_rate=0.5):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=hidden_layer_sizes[0], activation='relu', input_dim=X_train_scaled.shape[1]))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(units=hidden_layer_sizes[1], activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train_scaled, y_train, epochs=150, verbose=0)

    return model


# Define hyperparameters to search
hyperparameters = {
    'hidden_layer_sizes': [(128, 64), (128, 64, 32), (64, 32)],
    'alpha': [0.01, 0.001, 0.0001],
    'dropout_rate': [0.4, 0.5, 0.6]
}



In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer with 64 nodes and ReLU activation
nn.add(tf.keras.layers.Dense(units=256, activation='relu', input_dim=input_features))

# Second hidden layer with 32 nodes and ReLU activation
nn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer with one node and sigmoid activation for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               12032     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 45057 (176.00 KB)
Trainable params: 45057 (176.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5699 - accuracy: 0.7213
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5546 - accuracy: 0.7297
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7314
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5496 - accuracy: 0.7314
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7317
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7329
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accuracy: 0.7332
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5452 - accuracy: 0.7332
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5444 - accuracy: 0.7330
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5446 - accuracy: 0.7343

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5617 - accuracy: 0.7287 - 657ms/epoch - 2ms/step
Loss: 0.5617021322250366, Accuracy: 0.7287463545799255


In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer with 64 nodes and ReLU activation
nn.add(tf.keras.layers.Dense(units=512, activation='relu', input_dim=input_features))

# Second hidden layer with 32 nodes and ReLU activation
nn.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Output layer with one node and sigmoid activation for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               24064     
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dense_14 (Dense)            (None, 1)                 257       
                                                                 
Total params: 155649 (608.00 KB)
Trainable params: 155649 (608.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [34]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5355 - accuracy: 0.7384
Epoch 2/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5387 - accuracy: 0.7395
Epoch 3/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5340 - accuracy: 0.7388
Epoch 4/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5348 - accuracy: 0.7392
Epoch 5/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5342 - accuracy: 0.7397
Epoch 6/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5341 - accuracy: 0.7402
Epoch 7/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5338 - accuracy: 0.7400
Epoch 8/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5347 - accuracy: 0.7394
Epoch 9/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5345 - accuracy: 0.7397
Epoch 10/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5339 - accura

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6121 - accuracy: 0.7315 - 527ms/epoch - 2ms/step
Loss: 0.6120955348014832, Accuracy: 0.7315452098846436


In [36]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

c:\Users\cnlam\anaconda3.10\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
